In [38]:
import numpy as np
import pandas as pd
from quasarpy.quasar import Quasar, DatasetConfig, KrigingConfig

## 1. Generate Synthetic Data

We will use the same data as the integration test:
$$ y(t) = (2x_1 + x_2) \cdot (1 + 0.5t) $$

Where:
- $t \in [0, 2]$


In [39]:
# Define the functions from the integration test
def y1_func(t, x1, x2):
    return (2 * x1 + x2) * (1 + 0.5 * t)

def y2_func(t, x1, x2):
    return (3 * x1 - x2) * np.exp(-0.3 * t)

# Time steps
t_steps = np.linspace(0, 2, 21)

def generate_data_from_func(X_df, func, noise_std=0.0):
    y_data = {}
    for i in range(len(X_df)):
        row = X_df.iloc[i]
        curve = func(t_steps, row['x1'], row['x2'])
        if noise_std > 0:
            # Add Gaussian noise
            curve += np.random.normal(0, noise_std, size=curve.shape)
        y_data[i] = curve
    return pd.DataFrame(y_data).T

# Training Data (Same as test_quasar_integration)
X_train = pd.DataFrame({
    'x1': [1.0, 2.0, 3.0, 4.0, 5.0],
    'x2': [1.0, 2.0, 1.0, 2.0, 1.0]
})
Y_train_1 = generate_data_from_func(X_train, y1_func, noise_std=0.0)
Y_train_2 = generate_data_from_func(X_train, y2_func, noise_std=0.0)

print(f"Training Samples: {len(X_train)}")
print(X_train)

Training Samples: 5
    x1   x2
0  1.0  1.0
1  2.0  2.0
2  3.0  1.0
3  4.0  2.0
4  5.0  1.0


## 2. Train the Model

We configure a dataset and train the Quasar model.

In [40]:
# Configure Datasets
ds_config_1 = DatasetConfig(
    name='Dataset1',
    data=Y_train_1,
    kriging_config=KrigingConfig(basis_function=2) # Linear basis
)

ds_config_2 = DatasetConfig(
    name='Dataset2',
    data=Y_train_2,
    kriging_config=KrigingConfig(basis_function=2) # Linear basis
)

# Initialize Quasar
# Ensure ODYSSEE_CAE_INSTALLDIR is set in your environment
try:
    q = Quasar()
    print("Quasar initialized successfully.")
except FileNotFoundError as e:
    print(e)
    print("Please set ODYSSEE_CAE_INSTALLDIR to run this demo.")

# Train
if 'q' in locals():
    q.train(X_train, [ds_config_1, ds_config_2])
    print("Training complete.")

Quasar initialized successfully.
Training complete.


## 3. Validation

We generate a new set of data to validate the model's performance.

In [41]:
# Generate Validation Data (Intermediate points with Noise)
# Increase number of samples for better visualization
X_val = pd.DataFrame({
    'x1': np.random.uniform(0, 5, 20),
    'x2': np.random.uniform(0, 5, 20)
})

# Add noise to validation data to make plots interesting
# Amplitude is roughly 3 to 30, so noise of 1.0 is visible but not overwhelming
Y_val_1 = generate_data_from_func(X_val, y1_func, noise_std=1.5)
Y_val_2 = generate_data_from_func(X_val, y2_func, noise_std=0.5) # Smaller noise for smaller amplitude signal

ds_val_1 = DatasetConfig(
    name='Dataset1',
    data=Y_val_1
)

ds_val_2 = DatasetConfig(
    name='Dataset2',
    data=Y_val_2
)

# Run Validation
val_result = q.validate(X_val, [ds_val_1, ds_val_2])

# Show Metrics Summary
print(val_result.summary())

              RMSE       MAE  Peak Error     SRMSE
Dataset                                           
Dataset1  1.552928  1.224870    1.142118  0.287456
Dataset2  0.489674  0.388038    0.477692  0.125052


## 4. Interactive Dashboard

Use the dashboard below to inspect the results. 
- **Parity Plot**: Click on any point to see the corresponding curve comparison.
- **Curve Comparison**: Shows the Actual (Black) vs Predicted (Blue Dashed) curves.

In [42]:
val_result.dashboard()

## 5. Export Report

Save the validation results to an HTML file for sharing.

In [43]:
val_result.save_html('validation_report.html')
print('Report saved to validation_report.html')

Report saved to validation_report.html


## 6. Configuration Search

Find the optimal Kriging hyperparameters by searching through different combinations of basis functions, stationarity types, and nugget effects.

In [50]:
# Run config search with a subset of parameters for speed
cs_result = q.config_search(
    x=X_train,
    datasets=[ds_config_1, ds_config_2],
    x_val=X_val,
    val_datasets=[ds_val_1, ds_val_2],
    basis_functions=[1, 2],       # linear, quadratic, cubic
    stationarities=[3, 4],           # exp, matern32
    nugget_effects=[0.4, 0.8, 1.2]   # three nugget values
)

Config Search: 100%|██████████| 12/12 [00:06<00:00,  1.82config/s, bf=2, st=4, nug=1.2 | best SRMSE: 1.2505e-01]


In [ ]:
# Show summary
cs_result.summary(include_failures=cs_result.n_failed > 0).sort_values(['Dataset', 'SRMSE'])

,basis_function,stationarity,pulsation,nugget_effect,RMSE,MAE,Peak Error,SRMSE
Dataset,,,,,,,,
Dataset1,2,3,1.5708,0.4,1.552928,1.224870,1.142118,0.287456
Dataset1,2,3,1.5708,0.8,1.552928,1.224870,1.142118,0.287456
Dataset1,2,3,1.5708,1.2,1.552928,1.224870,1.142118,0.287456
Dataset1,2,4,1.5708,0.4,1.552928,1.224870,1.142118,0.287456
Dataset1,2,4,1.5708,0.8,1.552928,1.224870,1.142118,0.287456
Dataset1,2,4,1.5708,1.2,1.552928,1.224870,1.142118,0.287456
Dataset1,1,3,1.5708,0.4,3.345246,2.526132,3.076190,0.619224
Dataset1,1,4,1.5708,0.4,3.644368,2.792823,3.307605,0.674593
Dataset1,1,4,1.5708,0.8,3.942501,3.163088,3.825322,0.729779


In [52]:
# Get best config per dataset with custom weights
best_configs = cs_result.best(weights={'SRMSE': 1.0, 'MAE': 0.5})
print("Best configurations:")
for ds_name, config in best_configs.items():
    print(f"  {ds_name}: basis={config.basis_function}, stationarity={config.stationarity}, nugget={config.nugget_effect}")

Best configurations:
  Dataset1: basis=2, stationarity=3, nugget=0.4
  Dataset2: basis=2, stationarity=4, nugget=1.2


## 7. Interactive Dashboard with Weight Sliders

Use the dashboard below to explore configuration search results interactively.
- **Weight Sliders**: Adjust the importance of each metric (RMSE, MAE, Peak Error, SRMSE).
- **Bar Chart**: Configurations ranked by weighted score, with the best highlighted in gold.

In [55]:
cs_result.dashboard()

## 8. Export Config Search Report

Save the configuration search results to an HTML file with interactive weight sliders.

In [48]:
cs_result.save_html('config_search_report.html')
print('Config search report saved to config_search_report.html')

Config search report saved to config_search_report.html
